In [11]:
from google.colab import files
import pandas as pd

In [3]:
uploaded = files.upload()

Saving clean_data to clean_data


In [61]:
with open("clean_data", "r") as file:
    lines = file.readlines()
    
cleaned_tweet_list = [line.split() for line in lines]


In [60]:
cleaned_tweet=list(cleaned_tweet_list)
for i in range(len(cleaned_tweet)):
    cleaned_tweet[i] = ' '.join(cleaned_tweet[i])


In [37]:
# Transformer notre échantillon en dataframe
df = pd.DataFrame(cleaned_tweet,columns =['cleaned_tweet'])
print(df['cleaned_tweet'].head())

0    être avoir culot venir jouer victime mécher gr...
1    jslalsbd faisc taire veu dieuj llook effectvue...
2                                                unise
3    meme être fan puma aller vaincre milan ac team...
4                                  regarde truc savoir
Name: cleaned_tweet, dtype: object


## 6 - **Vectorization** (featurization)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

A- **Méthode BOW**

In [6]:
# définir le tokenizer
def tokenizer(text):
    return text


In [18]:
bow_vectorizer = CountVectorizer(analyzer='word',lowercase=False)
print('Tweet before vectorization: {}'.format(cleaned_tweet[83]))
bow = bow_vectorizer.fit_transform(cleaned_tweet)

print('Tweet after vectorization: \n{}'.format(bow[83]))
bow.shape

Tweet before vectorization: monsieur député attention souffrance peuple iranien démocratie pouvoir apporter paix pays cnri soutenir œuvre obtenir
Tweet after vectorization: 
  (0, 6727)	1
  (0, 8032)	1
  (0, 706)	1
  (0, 7611)	1
  (0, 3292)	1
  (0, 9671)	1
  (0, 7722)	1
  (0, 5313)	1
  (0, 3248)	1
  (0, 509)	1
  (0, 7442)	1
  (0, 1965)	1
  (0, 9700)	1
  (0, 11510)	1
  (0, 7214)	1


(5000, 11549)

In [17]:
#Mots les plus courants et leur nombre respectif
def get_most_freq_words(str, n=None):
    vect = CountVectorizer().fit(cleaned_tweet)
    bag_of_words = vect.transform(cleaned_tweet)
    sum_words = bag_of_words.sum(axis=0) 
    freq = [(word, sum_words[0, idx]) for word, idx in vect.vocabulary_.items()]
    freq =sorted(freq, key = lambda x: x[1], reverse=True)
    return freq[:n]
  
get_most_freq_words([ word for tweet in cleaned_tweet for word in tweet],20)

[('être', 2532),
 ('avoir', 1688),
 ('faire', 548),
 ('aller', 356),
 ('pouvoir', 249),
 ('bien', 196),
 ('voir', 184),
 ('savoir', 151),
 ('vouloir', 146),
 ('jour', 134),
 ('devoir', 120),
 ('france', 118),
 ('falloir', 117),
 ('gagner', 100),
 ('vie', 99),
 ('amp', 99),
 ('prendre', 96),
 ('follow', 93),
 ('grand', 91),
 ('taire', 88)]

J'essaie ici de jouer avec min_df et max_df de countvectorizer , et ça n'a pas l'air de marcher , car les tweets ont bien été nettoyé, de plus cela témoingne d'une grande diversité des mots au sein de notre corpus

In [24]:
#Mots les plus courants et leur nombre respectif
def get_most_freq_words(str, n=None):
    vect = CountVectorizer(max_df=0.3, min_df=5).fit(cleaned_tweet)
    bag_of_words = vect.transform(cleaned_tweet)
    sum_words = bag_of_words.sum(axis=0) 
    freq = [(word, sum_words[0, idx]) for word, idx in vect.vocabulary_.items()]
    freq =sorted(freq, key = lambda x: x[1], reverse=True)
    return freq[:n]
  
get_most_freq_words([ word for tweet in cleaned_tweet for word in tweet],20)

[('avoir', 1688),
 ('faire', 548),
 ('aller', 356),
 ('pouvoir', 249),
 ('bien', 196),
 ('voir', 184),
 ('savoir', 151),
 ('vouloir', 146),
 ('jour', 134),
 ('devoir', 120),
 ('france', 118),
 ('falloir', 117),
 ('gagner', 100),
 ('vie', 99),
 ('amp', 99),
 ('prendre', 96),
 ('follow', 93),
 ('grand', 91),
 ('taire', 88),
 ('covid', 88)]

On constate ici que les mots les plus courants avec BOW sont des verbes non conjugés tels que : être, faire, avoir, etc. 

B- **Méthode TFIDF**

In [25]:
#from numpy.lib.function_base import vectorize
#utiliser la méthode de Tfidfvectorizer avec:
""" - les stopsword français importés depuis spacy
    - le tokenizer définis plus haut"""
vectorizer = TfidfVectorizer(lowercase=False)
print('Tweet before vectorization: {}'.format(cleaned_tweet[83]))

Tweet before vectorization: monsieur député attention souffrance peuple iranien démocratie pouvoir apporter paix pays cnri soutenir œuvre obtenir


In [26]:
# transformer les tweets en vecteurs
# qui prend en argument les vecteurs de mots de l'ensemble des tweets nettoyés précédemment 
X = vectorizer.fit_transform(cleaned_tweet)
print('Tweet after vectorization: \n{}'.format(X[83]))

#print ((X.todense()))

Tweet after vectorization: 
  (0, 7214)	0.2635719556164827
  (0, 11510)	0.2635719556164827
  (0, 9700)	0.21467099683579413
  (0, 1965)	0.3177281701587722
  (0, 7442)	0.28473596007380847
  (0, 509)	0.28473596007380847
  (0, 3248)	0.2726208503905979
  (0, 5313)	0.3177281701587722
  (0, 7722)	0.22978836711560893
  (0, 9671)	0.28473596007380847
  (0, 3292)	0.25033155846161137
  (0, 7611)	0.19350699237846838
  (0, 706)	0.25634655268237083
  (0, 8032)	0.14751257521431016
  (0, 6727)	0.23482067313412833


In [27]:
idf=vectorizer.idf_

C- **Word embeding with Spacy**

In [28]:
def my_word_embedding(sentence):
    # Tokeniser la phrase
    doc = nlp(sentence)
    # Retourner le vecteur lié à chaque token
    return [(X.vector) for X in doc]

#for n, snt in enumerate(s):
    #print(n , ":", my_word_embedding(snt), "\n")

    print(X)

D- **Méthode W2Vec**

In [46]:
import gensim


#tokenized_tweet = s.apply(lambda x: x.split()) # tokenizing 

model_w2v = gensim.models.Word2Vec(
            cleaned_tweet_list,
            size=200, # desired no. of features/independent variables
            window=5, # context window size
            min_count=2, # Ignores all words with total frequency lower than 2.                                  
            sg = 1, # 1 for skip-gram model
            hs = 0,
            negative = 10, # for negative sampling
            workers= 32, # no.of cores
            seed = 34
) 


In [47]:
print(model_w2v)

Word2Vec(vocab=4641, size=200, alpha=0.025)


In [48]:
words = list(model_w2v.wv.vocab)
print(words)

['être', 'avoir', 'venir', 'jouer', 'victime', 'mécher', 'gros', 'compte', 'lancer', 'vague', 'abonné', 'insupportable', 'voir', 'faire', 'direct', 'taire', 'veu', 'mai', 'cest', 'mets', 'pression', 'rang', 'tablette', 'meme', 'fan', 'aller', 'vaincre', 'milan', 'ac', 'om', 'regarde', 'truc', 'savoir', 'file', 'minute', 'trottoir', 'quartier', 'alentours', 'bordeaux', 'chef', 'lieu', 'grand', 'région', 'administration', 'boîte', 'priver', 'no', 'es', 'devoir', 'mémoire', 'reprendre', 'vie', 'grâce', 'olivier', 'laurent', 'article', 'info', 'extrême', 'droite', 'rentrer', 'cas', 'reproche', 'gauche', 'préférer', 'soigner', 'aide', 'soignant', 'nonvacciné', 'perso', 'prendre', 'charge', 'manque', 'personnel', 'nonvacciner', 'hôpital', 'irresponsable', 'tchau', 'département', 'policier', 'intensifier', 'contrôle', 'passsanitair', 'établissement', 'recevoir', 'public', 'midi', 'centre', 'ville', 'lutter', 'covid', 'vue', 'saisissant', 'fort', 'mont', 'couleur', 'ilovenice', 'nice', 'coteda

In [49]:
#Nombre de mots dans le vocabulaire 

vocab_len = len(model_w2v.wv.vocab)
print(vocab_len)

4641


In [50]:
model_w2v['haine']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 6.24008887e-02, -4.01063561e-02, -5.81204779e-02,  2.59715542e-02,
        9.80506092e-02, -8.36496279e-02,  1.18035443e-01, -5.04827015e-02,
       -1.33620858e-01,  1.01981081e-01,  3.39286439e-02,  6.27613142e-02,
        3.07428949e-02, -2.15686619e-01, -1.53206259e-01,  3.99127938e-02,
       -5.44108301e-02,  1.17917672e-01, -5.00415899e-02,  1.07102133e-01,
        1.15598276e-01, -1.83084011e-02,  1.24082658e-02,  2.76882648e-01,
        1.51289599e-02,  7.71661401e-02,  2.74327416e-02,  1.33635588e-02,
        9.19034064e-04, -2.68162310e-01,  2.07554474e-02, -1.46591648e-01,
       -4.09845710e-02, -8.63641351e-02,  8.16857889e-02,  9.62626263e-02,
       -6.03522398e-02, -5.66906743e-02, -9.65556949e-02, -3.70929658e-01,
       -6.23883121e-02, -6.51803752e-03, -5.62752523e-02,  2.18960434e-01,
       -1.70065518e-02, -2.98029743e-02, -2.02314574e-02, -1.32267118e-01,
       -8.22221041e-02,  8.23857337e-02, -4.57786024e-02,  1.90132856e-02,
       -5.75115085e-02,  

In [59]:
model_w2v.wv.most_similar(positive="finir")

[('train', 0.9997788667678833),
 ('donner', 0.9997481107711792),
 ('connaître', 0.9997479319572449),
 ('mal', 0.9997469782829285),
 ('sortir', 0.9997106790542603),
 ('place', 0.9997091293334961),
 ('enfant', 0.999683141708374),
 ('arriver', 0.9996689558029175),
 ('moment', 0.9996660351753235),
 ('problème', 0.9996576905250549)]